# ICPC 2006 at UCSY

<http://www.ucsy.edu.mm/ucsy/pages/2016_AR_Yangon_Contest_Problems.pdf>


## How to solve Problem A (Evaluation Fully Parenthesized Expression)

```
Example: (1+(23*345))
```

### 1st step
Define a function that takes exact 3 parameters, 2nd parameter is operator ('+', '-', '\*', '/'), 1st and 3rd parameter are numeric value or List. If the parameter is List, call the function recursively.

In [ ]:
def eval_formula3(a, ope, b):
    '''
    Input: a,b numeric value or List of 3 element
           ope: '+', '-', '*', '/'
    Return evaluated value
    '''
    
    if isinstance(a, list):
        a = eval_formula3(a[0], a[1], a[2])
    if isinstance(b, list):
        b = eval_formula3(b[0], b[1], b[2])
        
    if ope == '+':
        return a+b
    elif ope == '-':
        return a-b
    elif ope == '*':
        return a*b
    elif ope == '/':
        return a/b
    else:  # invalid argument
        raise ValueError('invalid operator: ', ope)

In [ ]:
# Test eval_formula3
def test_eval_formula3():
    print('1st: ', eval_formula3(1, '+', 2))
    print('2nd: ', eval_formula3(1, '+', [23, '*', 345]))
    print('3rd: ', eval_formula3([2, '+', 3], '+', [[[2, '-', 1], '*', 2], '*', 5.0]))
    eval_formula3(2, 'x', 1)  # force error
    
#test_eval_formula3()

### 2nd step: Define eval_list()

Modify eval_formula3() to take a nuemeric value or List which contains 1 to 3 element. If the List contains 1 element, the element is numeric value or List, if the List contains 2 elements, first element shall be a sign ('+' or '-').

In [ ]:
def eval_list(aList):
    '''
    aList: nuemeric value(integer or float) or List
    return: evalueted nuemeric value
    '''
    
    if isinstance(aList, int) or isinstance(aList, float):
        return aList
    
    if len(aList) == 1:
        if isinstance(aList[0], list):
            return eval_list(aList[0])
        else:
            return aList[0]
        
    elif len(aList) == 2:
        if aList[0] == '+':
            return eval_list(aList[1])
        elif aList[0] == '-':
            return -1 * eval_list(aList[1])
        else:
            raise ValueError('shall be + or -', aList[0])
        
    elif len(aList) == 3:
        ope = aList[1]  # 2nd element is operator
        if ope == '+':
            return eval_list(aList[0]) + eval_list(aList[2])
        elif ope == '-':
            return eval_list(aList[0]) - eval_list(aList[2])
        elif ope == '*':
            return eval_list(aList[0]) * eval_list(aList[2])
        elif ope == '/':
            return eval_list(aList[0]) / eval_list(aList[2])
        else:
            raise ValueError('Invalid operator', ope)
            
    else:
        raise ValueError('Invalid argument to eval_list', aList)

In [ ]:
# test eval_list()
def test_eval_list():
    print('expect 1: ', eval_list(1))
    print('expect 2: ', eval_list([2]))
    print('expect 3: ', eval_list([[3]]))
    
    print('expect 4: ', eval_list(['+', 4]))
    print('expect -5: ', eval_list(['-', 5]))
    print('expect 6: ', eval_list(['-', ['-', 6]]))
    
    print('expect 2: ', eval_list([1, '+', 1]))
    print('expect 6: ', eval_list([[1, '+', 2], '+', 3]))
    print('expect -6: ', eval_list(['-', [[1, '+', 2], '+', 3]]))
    
    # error test
    #print(eval_list(['*', 3]))
    #print(eval_list([1, '$', 1]))
    #print(eval_list([1, '+', 1, '+', 1]))
    #print(eval_list([]))
    
# test_eval_list()

### 3rd step: Create Token List from Input String

```
'(1+(23*345)' -> ['1', '+', '(', '23', '*', '345', ')']
```

In [ ]:
def str2token(formula):
    '''
        input: String
        return: List of Token
    '''
    digit = ''
    token_list = list()

#    print(formula, file=sys.stderr)
    for c in formula:
        if c in '()+-*/':
            if digit != '':
                token_list.append(digit)
                digit = ''
            token_list.append(c)
        elif c in '0123456789.':
            digit += c
        elif c in '\n':
            if digit != '':
                token_list.append(digit)
                digit = ''
        else:
            raise ValueError

    if digit != '':
        token_list.append(digit)
#    print(token_list, file=sys.stderr)
    return token_list


In [ ]:
def test_str2token():
    print(str2token('(12+345)'))
    print(str2token('((1+1)*2)'))
    print(str2token('1.414*3.14'))
    
# test_str2token()

### 4th Step: Create nested List ( up to 3 element ) from Token List


```
['1', '+', '(', '23', '*', '345', ')'] -> [1.0, '+', [23.0, '*', 345.0] ]
```

Flat list of 7 elements -> Nested List of 3 elements (3rd element is a list of 3 element)  
Convert string of digits to float so that eval_list can handle it.

In [ ]:
def token2list(token_list):
    '''
        input: List of Token
        return: Up to 3 element list, each element may be a list
    '''
#    print('token2list:',token_list, file=sys.stderr)
    nested_list = list()
    p = 0
    while p < len(token_list):
        if token_list[p] == '(':
            open_p = 1
            for q in range(p+1,len(token_list)):
                if token_list[q] == ')':
                    open_p -= 1
                    if open_p == 0:   # matching close paren found
                        break
                elif token_list[q] == '(':
                    open_p += 1

            if open_p != 0:
                print('Improper input file, number of parentesis not match')
                raise ValueError

            nested_list.append(token2list(token_list[p+1:q]))
            p = q
        elif token_list[p] in '+-*/':
            nested_list.append(token_list[p])
        else:
            nested_list.append(float(token_list[p]))

        p += 1

#    print(nested_list, file=sys.stderr)
    return nested_list

In [ ]:
def test_token2list():
    print(token2list(str2token('1+1')))
    print(token2list(str2token('((-1)+2)+2')))
    
# test_token2list()

### Final Step: Handle input file and exception handling

Read input file (a.in), 1st line contains number of test case, 2nd and later are formula string to be evaluated.  
If **zero devide** is detected, print **Infinity**.

In [ ]:
infile = open('a.in', 'r')
num_test = int(infile.readline())

for i in range(num_test):
    fomula = infile.readline()
    token_list = str2token(fomula)
    nested_list = token2list(token_list)
    try:
        result = eval_list(nested_list)
    except ZeroDivisionError:
        print('Infinity')
    else:
        print(float(result))